# ACS AWS Training: Day 4
### Nathan Miles
06/11/2020

## Outline
- Review of last time
- Lambda
    - Recap
    - Anatomy of a Lambda Function
- Example
    - Create a lambda function using the console editor
    - Testing your lambda function in the console
    - Invoking the lambda function from your laptop via `boto3`

## Day 3 Review
- EC2 Launch Templates
    - Facilitate the repeated deployment of the same instance type
- AWS setup scripts
    - Shell scripts designed to minimize setup overhead when working with EC2 instances
- EC2 AMI
    - Obviates the need to install software 
 


## Review of Lambda
- "Serverless" web service
    - No need to specify anything about the physical infrastructure
    - Only specify the code you want to run
- The **runtime** of a Lambda function defines the execution environment for your code 
    - Python versions 2.7, 3.6, 3.7, 3.8
- To execute your code you **invoke** the Lambda function 
    - When invoked, AWS will assembles to computational resources required for your code to run

## Review of Lambda: Constraints
- Lambda is **not** meant to be used for everything
    - Suited for isolated tasks that require a short amount of time to run
        - Extracting photometry from a single image
        - Applying a correction to input data
            - ACS Photometric CTE REST API
        - Serving up filter throughput curves (requires data to exist in the cloud)
        
- There are constraints on the types of taks Lambda can handle
    - Memory allcation: Max of 3 GB
    - Execution Time: Max of 900 seconds
    - Payload: Max of 6 MB for synchronous, 256 KB for asynchronous
    - Deployment package:
        - 50 MB to 250 MB





## Anatomy of a Lambda Function
- The code is just a python module (e.g. `photometry.py`)
- It must have a function, known as the handler, that accepts the following arguments,
    - `event`
    - `context`
    
```python
    def handler_name(event, context):
                    .
                    .
```
- The name can be whatever you want, just make sure to indicate what it is when you create it
- Values returned will be serialized into `JSON` format

## Anatomy of a Lambda Function Cont.
- `event`
    - This object contains any input data pass to the handler
    - It is typically a Python `dict`, but it can be any of the following Python `dtypes`,
        - `list`, `str`, `int`, `float`, `NoneType`
    - This parameter must also be serialized into `JSON` format prior to being submitted to the lambda function
- `context`
    - This object contains methods and attributes that provide access to information about the invocation, function, and execution environment
    - Full details <a href="https://docs.aws.amazon.com/lambda/latest/dg/python-context.html">here</a>

## Example
- We are going to create a Lambda function that accepts a Python `dict`
```python
            event = {
                'first_name': 'Nathan',
                'last_name': 'Miles'
            }
```

- The lambda function will use this input to generate a message and return it
- We'll learn how to,
    - test the lambda function from the console
    - invoke the lambda function using `boto3`

## Example: Create the Lambda Function
- Login to the AWS Management Console
- Navigate to the AWS Lambda console
    - On the toolbar to the left, select Functions
    - Click Create Function
    - Select "Author from scratch"
- Function Configuration    
    - Function name: whatever you want
    - Runtime: `Python 3.6`
    - Chose or create an execution role:
        - Select "Use an Existing Role" --> `acs-aws-lambda`
- This will create a basic lambda function called `lambda_function.py`

## Example: Create the Lambda Function
- Navigate to the code editor for your Lambda function
- Copy and paste the code from the slide below
- When you're done, click save (upper right corner)

```python
import json
import logging

logging.basicConfig(format='%(levelname)-4s '
                           '[%(module)s.%(funcName)s:%(lineno)d]'
                           ' %(message)s',
                    )
LOG = logging.getLogger('Lambda')
LOG.setLevel(logging.INFO)

def func(event, context):
    message = 'Hello {} {}!'.format(event['first_name'], 
                                    event['last_name']) 
    return {
        'statusCode': 200,
        'body': json.dumps(message)
    }
```

## Example: Testing the Lambda Function
- Let's configure a "test event" to test our lambda function from console
- We are going to use the "hello-word" event template and update the following values
    - Change `key1` to `first_name`
    - Change `key2` to `last_name`
    - Delete `key3`

<img src="./lambda_test.png" width="75%" height="75%"/>

## Example: Invoking the Lambda with `boto3`
- Open an `ipython` session in your terminal
- Copy and paste the code in the slide below into your terminal

```python
import boto3
import json
lambda_name = 'Name of your lambda function' 
session = boto3.Session(profile_name='ndmiles_admin')
client = session.client('lambda', region_name='us-east-1')
event = {
    'first_name': 'Your first name',
    'last_name': 'Your last name'
}
Payload = json.dumps(event)
lambda_inputs = {
    'FunctionName': lambda_name,
    'InvocationType': 'RequestResponse',
    'LogType': 'Tail',
    'Payload': Payload
}
response = client.invoke(**lambda_inputs)
result = json.loads(response['Payload'].read())
print(result['body'])
```